In [1]:
import os
source_dir = "../Dataset_Willys_2020/ORGINAL/"
target_dir = os.listdir(source_dir)
print(target_dir)


['husdjur', 'fisk_skaldjur', 'apotek', 'mejeri_ost_agg', 'halsa_skonhet', 'skafferi', 'vegetariskt', 'barn', 'chicken_chark_meat', 'fardigmat', 'tidningar_bocker', 'frukt_gront', 'blommor', 'fryst', 'glass_godis_snacks', 'tradgard', 'hem_stad', 'brod_kakor']


In [2]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import AutoAugmentPolicy, AutoAugment,RandomApply   

polices = [AutoAugmentPolicy.CIFAR10,AutoAugmentPolicy.IMAGENET, AutoAugmentPolicy.SVHN]
augments = [AutoAugment(policy) for policy in polices]

applier = RandomApply(transforms=augments, p=[0.2, 0.4, 0.4])
dataset = ImageFolder(root=source_dir, transform=applier)


In [3]:
valid_split = 0.15
batch_size = 1
n_jobs = 12
n_epochs = 50

In [4]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import numpy as np

In [5]:
num_train = len(dataset)
indices = list(range(num_train))
split = int(valid_split*num_train)

valid_idx = np.random.choice(indices,size=split,replace=False)
train_idx = list(set(indices)-set(valid_idx))

train_sampler= SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size,
num_workers=n_jobs,pin_memory=True)
valid_loader = DataLoader(dataset,sampler=valid_sampler, batch_size=batch_size
, num_workers=n_jobs,pin_memory = True)

In [7]:
%%writefile load_data.py

from typing import Optional, Callable

from torchvision.transforms import AutoAugmentPolicy, AutoAugment, RandomApply
from torchvision.datasets import ImageFolder




def load_data_from_ImageFolder(root:str, transform:Optional[Callable]=None):
    """Takes path of the root directory for the images
    creates a dataset using pytorch ImageFolder class
    using AutoAugment for CIFAR,IMAGENET, SVHN with probability 
    p=[0.2, 0.4, 0.4] if transform is not provided.

    Args:
        root (str): path to the diroctroy where the images are separated into their classes by their folder
        transform (Optional): Applies transform for the images to increase the variaty.
    Return:
        ImageFolder dataset.
    
    """
    if transform is None:
        polices = [AutoAugmentPolicy.CIFAR10,
                   AutoAugmentPolicy.IMAGENET, AutoAugmentPolicy.SVHN]
        augments = [AutoAugment(policy) for policy in polices]
        applier = RandomApply(transforms=augments, p=[0.2, 0.4, 0.4])
    else:
        applier = transform
    dataset = ImageFolder(root=source_dir, transform=applier)
    return dataset


Writing load_data.py
